In [2]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import RANSACRegressor
from scipy.spatial import ConvexHull
import logging
from scipy.spatial.distance import pdist, squareform
import open3d as o3d
import numpy as np
from sklearn.linear_model import RANSACRegressor
from scipy.spatial import cKDTree
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# Load the segmented point cloud
pcd = o3d.io.read_point_cloud("/home/t/Desktop/work/results/combined_pointcloud.pcd")

# Visualize the point cloud to check its structure
o3d.visualization.draw_geometries([pcd])

# Print basic information about the point cloud
print(f"Points: {len(pcd.points)}")
print(f"Has normals: {pcd.has_normals()}")
print(f"Has colors: {pcd.has_colors()}")

Points: 93640
Has normals: True
Has colors: True


In [15]:
import open3d as o3d
import numpy as np
from sklearn.linear_model import RANSACRegressor
from scipy.spatial import cKDTree
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Load the segmented point cloud
pcd = o3d.io.read_point_cloud("/home/t/Desktop/work/results/combined_pointcloud1.pcd")

# Function to separate the point cloud into individual planes based on color
def separate_planes_by_color(pcd):
    colors = np.asarray(pcd.colors)
    unique_colors = np.unique(colors, axis=0)
    planes = []
    for color in unique_colors:
        color_mask = np.all(colors == color, axis=1)
        plane = pcd.select_by_index(np.where(color_mask)[0])
        planes.append(plane)
    return planes

# Separate the point cloud into individual planes
planes = separate_planes_by_color(pcd)

# Function to fit a plane to a set of points using RANSAC
def fit_plane_ransac(points):
    ransac = RANSACRegressor()
    X = points[:, :2]  # Use x, y for fitting
    y = points[:, 2]   # Use z as the target
    ransac.fit(X, y)
    normal = np.array([-ransac.estimator_.coef_[0], -ransac.estimator_.coef_[1], 1])
    normal /= np.linalg.norm(normal)
    centroid = np.mean(points, axis=0)
    return normal, centroid

# Function to find points close to both planes
def find_close_points(points1, points2, normal1, normal2, threshold=0.15):
    tree1 = cKDTree(points1)
    tree2 = cKDTree(points2)
    
    close_points = []
    for point in points1:
        dist, _ = tree2.query(point)
        if dist < threshold:
            # Check if point is close to both planes
            dist1 = np.abs(np.dot(point - points1[0], normal1))
            dist2 = np.abs(np.dot(point - points2[0], normal2))
            if dist1 < threshold and dist2 < threshold:
                close_points.append(point)
    
    return np.array(close_points)

# Function to fit a line to a set of points
def fit_line_to_points(points):
    if len(points) < 2:
        return None, None
    
    pca = PCA(n_components=1)
    pca.fit(points)
    
    direction = pca.components_[0]
    centroid = np.mean(points, axis=0)
    
    # Project points onto the line
    projected_points = np.dot(points - centroid, direction[:, np.newaxis]) * direction + centroid
    
    # Find the two extreme points
    distances = np.dot(projected_points - centroid, direction)
    min_idx, max_idx = np.argmin(distances), np.argmax(distances)
    
    return projected_points[min_idx], projected_points[max_idx]

# Find potential diffraction edges
def find_diffraction_edges(planes):
    diffraction_edges = []
    for i in range(len(planes)):
        for j in range(i + 1, len(planes)):
            points1 = np.asarray(planes[i].points)
            points2 = np.asarray(planes[j].points)
            
            normal1, _ = fit_plane_ransac(points1)
            normal2, _ = fit_plane_ransac(points2)
            
            close_points = find_close_points(points1, points2, normal1, normal2)
            if len(close_points) > 1:
                start_point, end_point = fit_line_to_points(close_points)
                if start_point is not None and end_point is not None:
                    diffraction_edges.append((start_point, end_point))
    
    return diffraction_edges

# Find potential diffraction edges
diffraction_edges = find_diffraction_edges(planes)

# Visualize diffraction edges
def visualize_diffraction_edges(diffraction_edges):
    edge_lines = []
    for start_point, end_point in diffraction_edges:
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector([start_point, end_point])
        line_set.lines = o3d.utility.Vector2iVector([[0, 1]])
        line_set.paint_uniform_color([1, 0, 0])  # Red color for diffraction edges
        edge_lines.append(line_set)
    
    o3d.visualization.draw_geometries(edge_lines + [pcd])

if diffraction_edges:
    visualize_diffraction_edges(diffraction_edges)
else:
    print("No diffraction edges found.")

In [19]:
import open3d as o3d
import numpy as np
from sklearn.linear_model import RANSACRegressor
from scipy.spatial import cKDTree
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Load the segmented point cloud
pcd = o3d.io.read_point_cloud("/home/t/Desktop/work/results/combined_pointcloud1.pcd")

# Function to separate the point cloud into individual planes based on color
def separate_planes_by_color(pcd):
    colors = np.asarray(pcd.colors)
    unique_colors = np.unique(colors, axis=0)
    planes = []
    for color in unique_colors:
        color_mask = np.all(colors == color, axis=1)
        plane = pcd.select_by_index(np.where(color_mask)[0])
        planes.append(plane)
    return planes

# Separate the point cloud into individual planes
planes = separate_planes_by_color(pcd)

def fit_plane_ransac(points, min_samples=5, residual_threshold=0.5, max_trials=1000):
    ransac = RANSACRegressor(
        min_samples=min_samples,
        residual_threshold=residual_threshold,
        max_trials=max_trials,
        random_state=42
    )
    X = points[:, :2]
    y = points[:, 2]
    ransac.fit(X, y)
    
    normal = np.array([-ransac.estimator_.coef_[0], -ransac.estimator_.coef_[1], 1])
    normal /= np.linalg.norm(normal)
    centroid = np.mean(points, axis=0)
    
    # Calculate plane quality metrics
    inliers = ransac.inlier_mask_
    inlier_ratio = np.sum(inliers) / len(points)
    
    return normal, centroid, inlier_ratio, inliers

# Enhanced close points detection
def find_close_points(points1, points2, normal1, normal2, threshold=0.08):  # Increased threshold
    """Find points that are close to both planes."""
    tree2 = cKDTree(points2)
    
    close_points = []
    for point in points1:
        dist, _ = tree2.query(point)
        if dist < threshold:
            # Check if point is close to both planes
            dist1 = np.abs(np.dot(point - points1[0], normal1))
            dist2 = np.abs(np.dot(point - points2[0], normal2))
            if dist1 < threshold * 1.5 and dist2 < threshold * 1.5:  # More lenient plane distance check
                close_points.append(point)
    
    return np.array(close_points)


# Enhanced line fitting with outlier removal
def fit_line_to_points(points, outlier_threshold=3.0):
    if len(points) < 2:
        return None, None
    
    pca = PCA(n_components=1)
    pca.fit(points)
    
    direction = pca.components_[0]
    centroid = np.mean(points, axis=0)
    
    # Project points onto the line
    projected_points = np.dot(points - centroid, direction[:, np.newaxis]) * direction + centroid
    
    # Remove outliers based on distance to the line
    distances = np.linalg.norm(points - projected_points, axis=1)
    inliers = distances < (np.mean(distances) + outlier_threshold * np.std(distances))
    
    if np.sum(inliers) < 2:
        return None, None
    
    # Use only inlier points for final line fitting
    clean_points = points[inliers]
    clean_projected = projected_points[inliers]
    
    # Find extreme points
    distances = np.dot(clean_projected - centroid, direction)
    min_idx, max_idx = np.argmin(distances), np.argmax(distances)
    
    return clean_projected[min_idx], clean_projected[max_idx]

# Enhanced edge detection
def find_diffraction_edges(planes, min_points=10, min_edge_length=0.05, max_edge_length=50.0):  # Adjusted parameters
    """Find potential diffraction edges between planes."""
    diffraction_edges = []
    edge_qualities = []
    
    # Filter out very small planes
    valid_planes = [(i, plane) for i, plane in enumerate(planes) if len(plane.points) > 150]  # Reduced minimum points
    
    for i, (idx1, plane1) in enumerate(valid_planes):
        for j, (idx2, plane2) in enumerate(valid_planes[i+1:], i+1):
            points1 = np.asarray(plane1.points)
            points2 = np.asarray(plane2.points)
            
            # Fit planes with more lenient parameters
            normal1, centroid1, quality1, inliers1 = fit_plane_ransac(
                points1,
                min_samples=3,
                residual_threshold=1.0,  # More lenient threshold
                max_trials=1000
            )
            normal2, centroid2, quality2, inliers2 = fit_plane_ransac(
                points2,
                min_samples=3,
                residual_threshold=1.0,
                max_trials=1000
            )
            
            # Calculate angle between planes
            angle = np.arccos(np.abs(np.dot(normal1, normal2))) * 180 / np.pi
            
            # More lenient angle check
            if angle < 5 or angle > 160:  # Adjusted angle thresholds
                continue
            
            # Find close points
            close_points = find_close_points(
                points1[inliers1],
                points2[inliers2],
                normal1,
                normal2
            )
            
            print(f"Found {len(close_points)} close points between planes {idx1} and {idx2}")
            
            if len(close_points) > min_points:
                # Fit line to close points
                pca = PCA(n_components=1)
                pca.fit(close_points)
                
                # Project points onto the line
                direction = pca.components_[0]
                centroid = np.mean(close_points, axis=0)
                projected = np.dot(close_points - centroid, direction[:, np.newaxis]) * direction + centroid
                
                # Find extreme points
                distances = np.dot(projected - centroid, direction)
                start_point = projected[np.argmin(distances)]
                end_point = projected[np.argmax(distances)]
                
                edge_length = np.linalg.norm(end_point - start_point)
                if min_edge_length <= edge_length <= max_edge_length:
                    # Calculate edge quality
                    point_density = len(close_points) / edge_length
                    edge_quality = point_density * (angle / 90)
                    
                    diffraction_edges.append((start_point, end_point))
                    edge_qualities.append(edge_quality)
                    
                    print(f"Edge found between planes {idx1} and {idx2}")
                    print(f"Length: {edge_length:.3f}, Angle: {angle:.1f}, Quality: {edge_quality:.3f}")
    
    # Keep only the top edges by quality
    if len(diffraction_edges) > 8:
        indices = np.argsort(edge_qualities)[-8:]
        diffraction_edges = [diffraction_edges[i] for i in indices]
    
    return diffraction_edges
    

# Enhanced visualization
def visualize_diffraction_edges(diffraction_edges, pcd):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    
    # Add point cloud
    vis.add_geometry(pcd)
    
    # Add edges with different colors based on confidence
    for i, (start_point, end_point) in enumerate(diffraction_edges):
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector([start_point, end_point])
        line_set.lines = o3d.utility.Vector2iVector([[0, 1]])
        
        # Color gradient from red (high confidence) to blue (lower confidence)
        color = plt.cm.RdYlBu(i / max(1, len(diffraction_edges) - 1))[:3]
        line_set.colors = o3d.utility.Vector3dVector([color])
        
        vis.add_geometry(line_set)
    
    # Optimize view
    vis.get_render_option().point_size = 1
    vis.get_render_option().line_width = 2
    vis.get_render_option().background_color = np.asarray([0.1, 0.1, 0.1])
    
    vis.run()
    vis.destroy_window()

# Main execution
print(f"Number of planes found: {len(planes)}")
for i, plane in enumerate(planes):
    print(f"Plane {i} has {len(plane.points)} points")

diffraction_edges = find_diffraction_edges(planes)

if diffraction_edges:
    print(f"Found {len(diffraction_edges)} potential diffraction edges")
    visualize_diffraction_edges(diffraction_edges, pcd)
else:
    print("No diffraction edges found")
    # Visualize planes for debugging
    for i, plane in enumerate(planes):
        color = plt.cm.tab10(i / len(planes))[:3]
        plane.paint_uniform_color(color)
    o3d.visualization.draw_geometries(planes)

Number of planes found: 12
Plane 0 has 4509 points
Plane 1 has 2874 points
Plane 2 has 13 points
Plane 3 has 252 points
Plane 4 has 2712 points
Plane 5 has 4338 points
Plane 6 has 6 points
Plane 7 has 22 points
Plane 8 has 651 points
Plane 9 has 3484 points
Plane 10 has 561 points
Plane 11 has 3139 points
Found 4 close points between planes 0 and 1
Found 0 close points between planes 0 and 3
Found 0 close points between planes 0 and 4
Found 0 close points between planes 0 and 8
Found 11 close points between planes 0 and 9
Edge found between planes 0 and 9
Length: 1.279, Angle: 89.9, Quality: 8.595
Found 0 close points between planes 0 and 10
Found 0 close points between planes 0 and 11
Found 0 close points between planes 1 and 3
Found 0 close points between planes 1 and 4
Found 0 close points between planes 1 and 5
Found 3 close points between planes 1 and 8
Found 22 close points between planes 1 and 9
Edge found between planes 1 and 9
Length: 0.818, Angle: 78.0, Quality: 23.312
Found 